In [ ]:
!pip3 install pandas dash

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


# Import required libraries
import pandas as pd
import plotly.express as px
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

lsites = spacex_df['Launch Site'].unique().tolist()
lsite = []
lsite.append({'label':'All Sites','value':'All Sites'})
for site in lsites:
    lsite.append({'label':site,'value':site})

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site_dropdown',...)
                                dcc.Dropdown(id='site_dropdown',
                                                options=lsite,
                                                value='All Sites',
                                                placeholder='Select a Launch Site Here',
                                                searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload_slider',...)
                                dcc.RangeSlider(id='payload_slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site_dropdown` as input, `success-pie-chart` as output
@app.callback(Output('success-pie-chart','figure'),
              Input('site_dropdown','value'))

def update_graph(site_dropdown):

    if (site_dropdown == 'All Sites'): 
        df = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names = 'Launch Site', title = '# of Successful Launches by Launch Site')
    else:
        df = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names = 'class', title = 'Outcomes of Launches at ' +site_dropdown)
    return fig

# TASK 4:
# Add a callback function for `site_dropdown` and `payload_slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output('success-payload-scatter-chart','figure'),
             Input('site_dropdown','value'),Input('payload_slider','value'))

def update_graph(site_dropdown,payload_slider):

    if (site_dropdown == 'All Sites'):
        fig = px.scatter(spacex_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',
                            range_x=payload_slider)
    else:
        df = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.scatter(df,x='Payload Mass (kg)',y='class',color='Booster Version Category',
                            range_x=payload_slider)
    return fig
        

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
